In [46]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [47]:
train_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

import pandas as pd
import numpy as np


In [48]:
# Define the threshold for removal
threshold = 0.7  # 70%

# Calculate the percentage of missing values for each column
missing_percentage_train = train_data.isnull().mean()
missing_percentage_test = test_data.isnull().mean()

# Drop columns with missing percentage greater than the threshold
columns_to_remove_train = missing_percentage_train[missing_percentage_train > threshold].index
columns_to_remove_test = missing_percentage_test[missing_percentage_test > threshold].index
train_data = train_data.drop(columns=columns_to_remove_train)
test_data = test_data.drop(columns=columns_to_remove_test)

In [49]:
# Data Cleaning and Preprocessing
train_data["SalePrice"] = np.log1p(train_data["SalePrice"])  # Apply log transformation to SalePrice

from sklearn.impute import SimpleImputer

# define imouter for numerical features
num_imputer = SimpleImputer(strategy='median')

# Impute missing values for numerical columns
train_data['LotFrontage'] = num_imputer.fit_transform(train_data[['LotFrontage']]).ravel()
train_data['GarageYrBlt'] = num_imputer.fit_transform(train_data[['GarageYrBlt']]).ravel()
test_data['LotFrontage'] = num_imputer.fit_transform(test_data[['LotFrontage']]).ravel()
test_data['GarageYrBlt'] = num_imputer.fit_transform(test_data[['GarageYrBlt']]).ravel()

# Impute MasVnrArea with 0
train_data['MasVnrArea'] = train_data['MasVnrArea'].fillna(0)
test_data['MasVnrArea'] = test_data['MasVnrArea'].fillna(0)

# Define imputer for categorical features
cat_imputer = SimpleImputer(strategy='most_frequent')
cat_imputer = SimpleImputer(strategy='most_frequent')

# Impute missing values for categorical columns and directly assign to the DataFrame
train_data['MasVnrType'] = cat_imputer.fit_transform(train_data[['MasVnrType']]).ravel()
train_data['BsmtQual'] = cat_imputer.fit_transform(train_data[['BsmtQual']]).ravel()
train_data['BsmtCond'] = cat_imputer.fit_transform(train_data[['BsmtCond']]).ravel()
train_data['BsmtExposure'] = cat_imputer.fit_transform(train_data[['BsmtExposure']]).ravel()
train_data['BsmtFinType1'] = cat_imputer.fit_transform(train_data[['BsmtFinType1']]).ravel()
train_data['BsmtFinType2'] = cat_imputer.fit_transform(train_data[['BsmtFinType2']]).ravel()
train_data['Electrical'] = cat_imputer.fit_transform(train_data[['Electrical']]).ravel()
train_data['FireplaceQu'] = cat_imputer.fit_transform(train_data[['FireplaceQu']]).ravel()
train_data['GarageType'] = cat_imputer.fit_transform(train_data[['GarageType']]).ravel()
train_data['GarageFinish'] = cat_imputer.fit_transform(train_data[['GarageFinish']]).ravel()
train_data['GarageQual'] = cat_imputer.fit_transform(train_data[['GarageQual']]).ravel()
train_data['GarageCond'] = cat_imputer.fit_transform(train_data[['GarageCond']]).ravel()


test_data['MasVnrType'] = cat_imputer.fit_transform(test_data[['MasVnrType']]).ravel()
test_data['BsmtQual'] = cat_imputer.fit_transform(test_data[['BsmtQual']]).ravel()
test_data['BsmtCond'] = cat_imputer.fit_transform(test_data[['BsmtCond']]).ravel()
test_data['BsmtExposure'] = cat_imputer.fit_transform(test_data[['BsmtExposure']]).ravel()
test_data['BsmtFinType1'] = cat_imputer.fit_transform(test_data[['BsmtFinType1']]).ravel()
test_data['BsmtFinType2'] = cat_imputer.fit_transform(test_data[['BsmtFinType2']]).ravel()
test_data['Electrical'] = cat_imputer.fit_transform(test_data[['Electrical']]).ravel()
test_data['FireplaceQu'] = cat_imputer.fit_transform(test_data[['FireplaceQu']]).ravel()
test_data['GarageType'] = cat_imputer.fit_transform(test_data[['GarageType']]).ravel()
test_data['GarageFinish'] = cat_imputer.fit_transform(test_data[['GarageFinish']]).ravel()
test_data['GarageQual'] = cat_imputer.fit_transform(test_data[['GarageQual']]).ravel()
test_data['GarageCond'] = cat_imputer.fit_transform(test_data[['GarageCond']]).ravel()

In [50]:
# new features engeneering train_data

# Total Square Footage to capture the overall livable space in the property
train_data['TotalSF'] = train_data['TotalBsmtSF'] + train_data['1stFlrSF'] + train_data['2ndFlrSF'] + train_data['GrLivArea']

# Age of the Property helps to capture the effect of older vs.newer properties on sale prices
train_data['HouseAge'] = train_data['YrSold'] - train_data['YearBuilt']

# Time Since Last Renovation
train_data['TimeSinceRemodel'] = train_data['YrSold'] - train_data['YearRemodAdd']

# Is Remodeled: binary feature indicates whether a home has undergone any renovations: IsRemodeled = 1 if YearBuilt != YearRemodAdd else 0
train_data['IsRemodeled'] = (train_data['YearBuilt'] != train_data['YearRemodAdd']).astype(int)

# Total Bathrooms
train_data['Total Bathrooms'] = train_data['FullBath'] + (train_data['HalfBath']*0.5) + train_data['BsmtFullBath'] + (train_data['BsmtHalfBath']*0.5)

# Has Basement
train_data['HasBasement'] = (train_data['TotalBsmtSF'] > 0).astype(int)

# Lot Frontage/Area Ratio: to see if the frontage size relative to the lot area has an effect on the house price
train_data['LotFrontageAreaRatio'] = train_data['LotFrontage'] / train_data['LotArea']

# Quality and Condition Interaction: to capture the combine impact of qual and cond
#train_data['QualCondIntercation'] = train_data['OverallQual'] * train_data['OverallCond']

# Seasonality of Sale
# Seasonality of Sale with unique labels
train_data['Season'] = pd.cut(
    train_data['MoSold'],
    bins=[0, 3, 6, 9, 12],
    labels=['Winter', 'Spring', 'Summer', 'Fall'],
    right=False,                        # right=False means the interval is closed on the left side, open on the right side: the interval [0, 3) means it includes 0 (December) up to but not including 3 (March)
    include_lowest=True                 # ensures that the lowest value (0 in this case) is included in the first bin.  
)

# Garage Feature Interaction: create a single feature that captures the quality and the size of the garage
train_data['GarFeatInter'] = train_data['GarageCars'] * train_data['GarageArea'] * (train_data['GarageFinish'].notnull().astype(int))

# Total Porch Area combine all porch-related features into one
train_data['TotPorchSF'] = train_data['OpenPorchSF'] + train_data['EnclosedPorch'] + train_data['3SsnPorch'] + train_data['ScreenPorch']

# Dropping old features that are replaced by new ones
train_data = train_data.drop(columns=['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 
                                      'FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath', 
                                      '1stFlrSF', '2ndFlrSF', 'TotalBsmtSF', 'GrLivArea',
                                      'GarageCars', 'GarageArea'
                                     ])

In [51]:
# new features engeneering test_data

# Total Square Footage to capture the overall livable space in the property
test_data['TotalSF'] = test_data['TotalBsmtSF'] + test_data['1stFlrSF'] + test_data['2ndFlrSF'] + test_data['GrLivArea']

# Age of the Property helps to capture the effect of older vs.newer properties on sale prices
test_data['HouseAge'] = test_data['YrSold'] - test_data['YearBuilt']

# Time Since Last Renovation
test_data['TimeSinceRemodel'] = test_data['YrSold'] - test_data['YearRemodAdd']

# Is Remodeled: binary feature indicates whether a home has undergone any renovations: IsRemodeled = 1 if YearBuilt != YearRemodAdd else 0
test_data['IsRemodeled'] = (test_data['YearBuilt'] != test_data['YearRemodAdd']).astype(int)

# Total Bathrooms
test_data['Total Bathrooms'] = test_data['FullBath'] + (test_data['HalfBath']*0.5) + test_data['BsmtFullBath'] + (test_data['BsmtHalfBath']*0.5)

# Has Basement
test_data['HasBasement'] = (test_data['TotalBsmtSF'] > 0).astype(int)

# Lot Frontage/Area Ratio: to see if the frontage size relative to the lot area has an effect on the house price
test_data['LotFrontageAreaRatio'] = test_data['LotFrontage'] / test_data['LotArea']

# Quality and Condition Interaction: to capture the combine impact of qual and cond
#test_data['QualCondIntercation'] = test_data['OverallQual'] * test_data['OverallCond']

# Seasonality of Sale
# Seasonality of Sale with unique labels
test_data['Season'] = pd.cut(
    test_data['MoSold'],
    bins=[0, 3, 6, 9, 12],
    labels=['Winter', 'Spring', 'Summer', 'Fall'],
    right=False,                        # right=False means the interval is closed on the left side, open on the right side: the interval [0, 3) means it includes 0 (December) up to but not including 3 (March)
    include_lowest=True                 # ensures that the lowest value (0 in this case) is included in the first bin.  
)

# Garage Feature Interaction: create a single feature that captures the quality and the size of the garage
test_data['GarFeatInter'] = test_data['GarageCars'] * test_data['GarageArea'] * (test_data['GarageFinish'].notnull().astype(int))

# Total Porch Area combine all porch-related features into one
test_data['TotPorchSF'] = test_data['OpenPorchSF'] + test_data['EnclosedPorch'] + test_data['3SsnPorch'] + test_data['ScreenPorch']

# Dropping old features that are replaced by new ones
test_data = test_data.drop(columns=['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 
                                      'FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath', 
                                      '1stFlrSF', '2ndFlrSF', 'TotalBsmtSF', 'GrLivArea',
                                      'GarageCars', 'GarageArea'
                                    ])

In [52]:
# Encode categorical columns train_data

from sklearn.preprocessing import LabelEncoder

# label encoding for ordinal features
ordinal_cols = ['ExterQual', 'ExterCond', 'HeatingQC', 'KitchenQual', 'Functional',
                'GarageQual', 'GarageCond', 'PavedDrive', 'BsmtQual', 'BsmtCond', 
                'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Condition2']

label_encoders = {}
for col in ordinal_cols:
    le = LabelEncoder()
    train_data[col] = le.fit_transform(train_data[col])
    label_encoders[col] = le

# one-hot encoding for nominal features
nominal_cols = ['MSZoning', 'LotShape', 'LandContour', 'Utilities', 'Neighborhood',
                'Condition1', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl',
                'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 
                'CentralAir', 'GarageType', 'SaleType', 'SaleCondition', 
                'Season', 'Street', 'LotConfig', 'LandSlope', 'Heating', 
                'Electrical', 'FireplaceQu', 'GarageFinish']


train_data = pd.get_dummies(train_data, columns = nominal_cols, drop_first=True)

In [53]:
# Encode categorical columns test_data

from sklearn.preprocessing import LabelEncoder

# label encoding for ordinal features
ordinal_cols = ['ExterQual', 'ExterCond', 'HeatingQC', 'KitchenQual', 'Functional',
                'GarageQual', 'GarageCond', 'PavedDrive', 'BsmtQual', 'BsmtCond', 
                'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Condition2']

label_encoders = {}
for col in ordinal_cols:
    le = LabelEncoder()
    test_data[col] = le.fit_transform(test_data[col])
    label_encoders[col] = le

# one-hot encoding for nominal features
nominal_cols = ['MSZoning', 'LotShape', 'LandContour', 'Utilities', 'Neighborhood',
                'Condition1', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl',
                'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 
                'CentralAir', 'GarageType', 'SaleType', 'SaleCondition', 
                'Season', 'Street', 'LotConfig', 'LandSlope', 'Heating', 
                'Electrical', 'FireplaceQu', 'GarageFinish']


test_data = pd.get_dummies(test_data, columns = nominal_cols, drop_first=True)

# filling NA values of test data
test_data.fillna(test_data.mean(), inplace=True)

In [54]:
# Align train_data and test_data to have the same columns
train_data, test_data = train_data.align(test_data, join='left', axis=1, fill_value=0)

# Split data into X (features) and y (target)
X_train = train_data.drop("SalePrice", axis=1)
y_train = train_data["SalePrice"]
X_test = test_data.drop("SalePrice", axis=1)  # Test set doesn't have 'SalePrice', but it must have same features

from sklearn.ensemble import RandomForestRegressor

# Model Training using the best hyperparameters
model = RandomForestRegressor(
    n_estimators=300,
    max_depth=30,
    min_samples_split=2,
    min_samples_leaf=3,
    max_features=0.3,
    bootstrap=False
)

model.fit(X_train, y_train)

# Predict for test set
test_preds = model.predict(X_test)

# Apply reverse log transformation to predicted prices for submission
final_preds = np.expm1(test_preds)  # Reverse log1p to get actual prices


In [59]:
# Create Submission File
submission = pd.DataFrame({
    "Id": test_data["Id"],
    "SalePrice": final_preds
})

# Save to CSV (ensuring a clear path)
submission_file_path = '/kaggle/working/submission.csv'  # Update with correct path
submission.to_csv(submission_file_path, index=False)

print(f"Submission file saved at {submission_file_path}")

Submission file saved at /kaggle/working/submission.csv
